# CS280 Programming Assignment 3
__Comparison of Boosted Perceptrons and SVM__<br>
<br>
Compiler: Python 3.6.5<br>
OS: Windows 7

## 2. Perceptron Classifier Construction

### 2.1. Generate data
Generate data consisting of 100 two-dimensional vectors taken from a normal distribution with $\mu_{1} = [0,0]^{T}$, $\sigma_{1} = I$ and label them as class "$-1$"<br>
<br>
Form a class "$-1$" training subset by setting aside 50 points and let the remaining 50 points serve as part of the test set.

In [1]:
import numpy as np

def generate_random_data(mean, standard_dev, data_size, train_test_split, label):
    data = np.random.normal(mean, standard_dev, size=data_size)
    
    train_data = data[ :int(train_test_split*data.shape[0])]
    train_labels = label * np.ones(train_data.shape[0])
    
    test_data = data[int(train_test_split*data.shape[0]): ]
    test_labels = label * np.ones(test_data.shape[0])
    
    return train_data, train_labels, test_data, test_labels
    

In [2]:
NUM_DATA_POINTS = 100
NUM_DATA_DIM = 2

mean = 0
std = 1

classneg1_train_data, classneg1_train_labels, classneg1_test_data, classneg1_test_labels = generate_random_data(mean, std, [NUM_DATA_POINTS, NUM_DATA_DIM], 0.5, -1)

Do the same for class "+1" for another normal distribution with $\mu_{2} = [0,0]^{T}$, $\sigma_{2} = I$. <br>

In [3]:
classpos1_train_data, classpos1_train_labels, classpos1_test_data, classpos1_test_labels = generate_random_data(mean, std, [NUM_DATA_POINTS, NUM_DATA_DIM], 0.5, 1)

Merge the subsets to form the training and test sets.

In [4]:
def shuffle_data(data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices, :]
    labels = labels[indices]
    return data, labels


In [5]:
train_set, train_labels = shuffle_data(np.concatenate((classneg1_train_data, classpos1_train_data)), np.concatenate((classneg1_train_labels, classpos1_train_labels)))
test_set, test_labels = shuffle_data(np.concatenate((classneg1_test_data, classpos1_test_data)), np.concatenate((classneg1_test_labels, classpos1_test_labels)))

### 2.2. Write code _classify(...)_
Write code named _classify(...)_ that implements the Pocket Algorithm for perceptron learning.<br>
Set _maxitercnt_ to 10000 iterations.

In [6]:
import random

def classify(train_set, train_labels, maxitercnt=10000):
    # Number of features, d
    d = train_set.shape[1]

    # Number of training samples
    N = train_set.shape[0]

    # Number of consecutive iterations for which v correctly classified the examples
    n_v = 0

    # Number of consecutive iterations for which w correctly classified the examples
    n_w = 0

    w = np.random.uniform(low=-0.1, high=0.1, size=(d+1, 1))
    v = w
    print("\t'-'-'-'-'-'-'-'-'-'-'-'-'-'-")
    print('\tPocket Algorithm training started:\n\t\t', end='')
    for itercnt in range(maxitercnt):
        if itercnt % int(maxitercnt/20) == 0:
            print('.', end='')
        random_index = random.sample(range(N), 1)
        x_j = np.insert(train_set[random_index], 0, 1)
        y_j = train_labels[random_index]

        y_j_hat = 1 if (np.inner(v.transpose(), x_j) >= 0) else -1

        if y_j*y_j_hat > 0:
            n_v += 1
            #print('\t\tweight vector v correctly predicted label!')
        else:
            #print('\t\tweight vector v incorrectly predicted label! Putting in pocket...')
            if n_v > n_w:
                #print('\n\t\t@Pocket Algorithm Iteration # %d of %d: FOUND NEW BEST W with n_w=%d!\n\t\t' % (itercnt+1, maxitercnt, n_w), end='')
                w = v
                n_w = n_v
            v = v + (y_j*x_j).reshape(-1, 1)
            n_v = 0
        
    print('\n\tPocket Algorithm training ended with n_w=%d, w=:' % (n_w), w.transpose() )
    print("\t'-'-'-'-'-'-'-'-'-'-'-'-'-'-")
        
    return w

In [7]:
maxitercnt = 10000
pocket_w = classify(train_set, train_labels, maxitercnt)

	'-'-'-'-'-'-'-'-'-'-'-'-'-'-
	Pocket Algorithm training started:
		....................
	Pocket Algorithm training ended with n_w=19, w=: [[ 0.09442872 -0.5047732   2.68514097]]
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-


### 2.3 Write code _predict()_
Write code named _predict(...)_ to test the classifier and  measure the sum of square errors for the test set.

In [8]:
def predict(input_data, w, true_labels=None):
    x_j = np.insert(input_data, 0, 1, axis=1)
    error = 0
    
    predicted_labels = np.zeros((input_data.shape[0], 1))
    for index in range(x_j.shape[0]):
        prod = np.inner(w.transpose(), x_j[index, :].reshape(1,-1))
        predicted_labels[index] = 1 if (prod >= 0) else -1
        
    if true_labels is not None:    
        error = (1/len(true_labels))*np.sum(np.square(true_labels - predicted_labels))
    
    return predicted_labels, error

In [9]:
predicted_labels, error = predict(test_set, pocket_w, test_labels)

In [10]:
error

200.0

## 3. Adaboost Construction and Evaluation


### 3.1. Write code _adabtrain()_
Write code called _adabtrain()_ that implements the Adaboost algorithm with the Pocket Algorithm as the basic learner.

In [19]:
import numpy as np

def adabtrain(data, labels, K=10):
    N = data.shape[0]
    w = np.full((data.shape[0],1), 1/N)
    alpha = np.zeros(K)
    h = np.zeros((K, data.shape[1]+1))
    print('w.shape = ', len(w), ' h.shape = ', h.shape, ' alpha.shape = ', alpha.shape)
    
    for iter in range(K):
        print("=======================================================")
        print("ADABOOST ITERATION # %d of %d" % (iter+1, K))
        print("=======================================================")

        eps = 1
        y_t = np.zeros(labels.shape)
        L_weights = np.zeros((data.shape[1]+1))
        h_t_of_x = np.zeros(labels.shape)
        while(eps >= 0.5):
            s_t_indices = np.random.choice(np.arange(labels.size), size=labels.shape, replace=True, p=w.transpose().tolist()[0])
            y_t = labels[s_t_indices]
            L_weights = classify(data[s_t_indices], y_t)

            h_t_of_x, _ = predict(data[s_t_indices], L_weights)

            delta = np.not_equal(labels.reshape(-1, 1), h_t_of_x)
            eps = np.sum(w*delta)

            print('\n\tADABOOST training_error = ', eps)
            if eps >= 0.5:
                print('\n\t!! ADABOOST training_error is >= 0.5! Re-sampling.... !!\n')
                iter -= 1
    
        alpha_t = (0.5)*np.log((1-eps)/(eps))
        
        def update_w(w_t, alpha_t, y_s_t, h_t):
            new_w = w_t*(np.exp(-alpha_t*y_s_t*h_t.transpose())).transpose()
            z_t = np.sum(new_w)
            
            new_w = new_w/z_t
            return new_w
        
        w = update_w(w, alpha_t, y_t, h_t_of_x)
        h[iter] = list(L_weights)
        alpha[iter] = alpha_t
        
    return 
    

### 3.2. Form the training set
Form the training set by sampling 400 points from the banana dataset provided. Use the remaining 4900 points as your test set.

In [12]:
raw_data = np.genfromtxt('banana_data.csv', delimiter=',')
all_data = np.genfromtxt('banana_data.csv', delimiter=',')[ : , 1:]
all_labels = np.genfromtxt('banana_data.csv', delimiter=',')[ : , 0]


In [13]:
NUM_TRAIN_POINTS = 400
NUM_TEST_POINTS = 4900

train_indices = np.random.choice(all_data.shape[0], NUM_TRAIN_POINTS, replace=False)
test_indices = np.array([x for x in np.arange(all_data.shape[0]) if x not in train_indices])

train_data = all_data[train_indices]
train_labels = all_labels[train_indices]

test_data = all_data[test_indices]
test_labels = all_labels[test_indices]

train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

((400, 2), (400,), (4900, 2), (4900,))

### 3.3. Write code named _adabpredict()_
Write code named adabpredict() that classifies unknown/unseen data

### 3.4. Train the boosted perceptron algorithm
Using _adabtrain()_ and _adabpredict()_, train the boosted perceptron algorithm and measure the training and test accuracies for the ensemble classifier having *K* learners.

In [20]:
adabtrain(train_data,train_labels)

w.shape =  400  h.shape =  (10, 3)  alpha.shape =  (10,)
ADABOOST ITERATION # 1 of 10
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-
	Pocket Algorithm training started:
		....................
	Pocket Algorithm training ended with n_w=13, w=: [[ 0.00741679 -0.2337937  -2.30477835]]
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-

	ADABOOST training_error =  0.47250000000000003
ADABOOST ITERATION # 2 of 10
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-
	Pocket Algorithm training started:
		....................
	Pocket Algorithm training ended with n_w=13, w=: [[ 0.03413976 -0.0720089  -2.25249663]]
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-

	ADABOOST training_error =  0.5369233489970189
	
!! ADABOOST training_error is >= 0.5! Re-sampling.... !!

	'-'-'-'-'-'-'-'-'-'-'-'-'-'-
	Pocket Algorithm training started:
		....................
	Pocket Algorithm training ended with n_w=13, w=: [[ 0.91729303 -0.89865653 -1.70120578]]
	'-'-'-'-'-'-'-'-'-'-'-'-'-'-

	ADABOOST training_error =  0.5044969935829418
	
!! ADABOOST training_error is >= 0.5! Re-sampling..

### 3.5. Write code that automatically plots accuracies
Write code that automatially plots the training and test accuracies against the number of learners *K* used for training. Use *K*= 10, 20, 30, ...., 1000

### 3.6. Train and test your Boosted Perceptron on the splice dataset
Train and test you Boosted perceptron on the splice dataset and plot the accuracy vs _K_ curves for both training and test. Use 1000 points for training, 2175 points for test, and K = 10, 20, 30, ..., 1000

## SVM Classifier
You have been provided LibSVM. Familiarize yourself with the codes. Compile it and test the codes.

Form a training dataset of 400 points and test set of 4900 points from banana dataset.<br>
<br>
Run the Support Vector Machine Classifier on these sets. Find the kernel and kernal parameters to obtain the best performance.<br>
<br>
Repeat for the splice dataset with 1000 training points and 2175 test points.<br>
<br>
Compare the performance of SVM with Boosted Perceptrons in terms of test accuracy, training and test speeds.